### Content Based Recommendations

In the previous notebook, you were introduced to a way of making recommendations using collaborative filtering.  However, using this technique there are a large number of users who were left without any recommendations at all.  Other users were left with fewer than the ten recommendations that were set up by our function to retrieve....

In order to help these users out, let's try another technique: **content based** recommendations. Let's start off where we were in the previous notebook.

In [1]:
!pip install progressbar

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import HTML
import progressbar
import tests as t
import pickle


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']


all_recs = pickle.load(open("all_recs.p", "rb"))

### Datasets

From the above, you now have access to three important items that you will be using throughout the rest of this notebook.  

`a.` **movies** - a dataframe of all of the movies in the dataset along with other content related information about the movies (genre and date)


`b.` **reviews** - this was the main dataframe used before for collaborative filtering, as it contains all of the interactions between users and movies.


`c.` **all_recs** - a dictionary where each key is a user, and the value is a list of movie recommendations based on collaborative filtering

For the individuals in **all_recs** who did receive 10 recommendations using collaborative filtering, we don't really need to worry about them.  However, there were a number of individuals in our dataset who did not receive any recommendations.

-----

`1.` Let's start with finding all of the users in our dataset who didn't get all 10 ratings we would have liked them to have using collaborative filtering.  

In [108]:
all_users

array([    1,     2,     3, ..., 53966, 53967, 53968])

In [111]:
a = set([1,2,3,4,5])
b = set([2,4])
a.difference(b)

{1, 3, 5}

In [113]:
users_with_all_recs = []
for user_id in all_recs.keys():
    if len(all_recs[user_id]) >= 10:
        users_with_all_recs.append(user_id)

all_users = reviews['user_id'].unique()        
        
# Store user ids who have all their recommendations in this (10 or more)
# Store users who still need recommendations here
users_who_need_recs = set(all_users).difference(set(users_with_all_recs))

In [119]:
len(users_with_all_recs)

22187

In [120]:
len(users_who_need_recs)

31781

In [121]:
# A quick test
assert len(users_with_all_recs) == 22187
print("That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!")

That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!


### Content Based Recommendations

You will be doing a bit of a mix of content and collaborative filtering to make recommendations for the users this time.  This will allow you to obtain recommendations in many cases where we didn't make recommendations earlier.     

`2.` Before finding recommendations, rank the user's ratings from highest to lowest. You will move through the movies in this order looking for other similar movies.

In [31]:
# create a dataframe similar to reviews, but ranked by rating for each user
reviews_ranked = reviews.sort_values(by=['user_id', 'rating'], ascending=[True,False])
reviews_ranked.head(100)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
16,2,1798709,10,1389948338,2014-01-17 08:45:38,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
18,2,2024544,10,1389355949,2014-01-10 12:12:29,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
22,2,2726560,9,1438293988,2015-07-30 22:06:28,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,2,816711,8,1379963769,2013-09-23 19:16:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9,2,1390411,8,1451374513,2015-12-29 07:35:13,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
10,2,1398426,8,1456160307,2016-02-22 16:58:27,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Similarities

In the collaborative filtering sections, you became quite familiar with different methods of determining the similarity (or distance) of two users.  We can perform similarities based on content in much the same way.  

In many cases, it turns out that one of the fastest ways we can find out how similar items are to one another (when our matrix isn't totally sparse like it was in the earlier section) is by simply using matrix multiplication.  If you are not familiar with this, an explanation is available [here by 3blue1brown](https://www.youtube.com/watch?v=LyGKycYT2v0) and another quick explanation is provided [in the post here](https://math.stackexchange.com/questions/689022/how-does-the-dot-product-determine-similarity).

For us to pull out a matrix that describes the movies in our dataframe in terms of content, we might just use the indicator variables related to **year** and **genre** for our movies.  

Then we can obtain a matrix of how similar movies are to one another by taking the dot product of this matrix with itself.  Notice below that the dot product where our 1 values overlap gives a value of 2 indicating higher similarity.  In the second dot product, the 1 values don't match up.  This leads to a dot product of 0 indicating lower similarity.

<img src="images/dotprod1.png" alt="Dot Product" height="500" width="500">

We can perform the dot product on a matrix of movies with content characteristics to provide a movie by movie matrix where each cell is an indication of how similar two movies are to one another.  In the below image, you can see that movies 1 and 8 are most similar, movies 2 and 8 are most similar, and movies 3 and 9 are most similar for this subset of the data.  The diagonal elements of the matrix will contain the similarity of a movie with itself, which will be the largest possible similarity (and will also be the number of 1's in the movie row within the orginal movie content matrix).

<img src="images/moviemat.png" alt="Dot Product" height="500" width="500">


`3.` Create a numpy array that is a matrix of indicator variables related to year (by century) and movie genres by movie.  Perform the dot product of this matrix with itself (transposed) to obtain a similarity matrix of each movie with every other movie.  The final matrix should be 31245 x 31245.

In [19]:
# Subset so movie_content is only using the dummy variables for each genre and the 3 century based year dummy columns
movie_content = movies.drop(['movie_id', 'movie', 'genre', 'date'], axis=1)
movie_content.head()

,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
# Take the dot product to obtain a movie x movie matrix of similarities
movie_matrix = np.dot(movie_content, movie_content.T)

In [21]:
# create checks for the dot product matrix
assert movie_matrix.shape[0] == 31245
assert movie_matrix.shape[1] == 31245
assert movie_matrix[0, 0] == np.max(movie_matrix[0])
print("Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!")

Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!


### For Each User...


Now you have a matrix where each user has their ratings ordered.  You also have a second matrix where movies are each axis, and the matrix entries are larger where the two movies are more similar and smaller where the two movies are dissimilar.  This matrix is a measure of content similarity. Therefore, it is time to get to the fun part.

For each user, we will perform the following:

    i. For each movie, find the movies that are most similar that the user hasn't seen.

    ii. Continue through the available, rated movies until 10 recommendations or until there are no additional movies.

As a final note, you may need to adjust the criteria for 'most similar' to obtain 10 recommendations.  As a first pass, I used only movies with the highest possible similarity to one another as similar enough to add as a recommendation.

`3.` In the cell below, complete each of the functions needed for making content based recommendations.

In [55]:
movie_matrix_df = pd.DataFrame(movie_matrix)

In [68]:
similar = movie_matrix_df.loc[movie_id, movie_matrix_df.loc[movie_id] != 0] \
    .sort_values(ascending=False)

In [83]:
widgets = ['Test: ', progressbar.Percentage(), ' ', progressbar.Bar(marker='0',left='[',right=']'),
           ' ', ETA()] 

pbar = ProgressBar(widgets=widgets, maxval=500)
pbar.start()

for i in range(100,500+1,50):
    # here do something long at each iteration
    pbar.update(i) #this adds a little symbol at each iteration
pbar.finish()

Test: 100% [0000000000000000000000000000000000000000000000000000] Time: 0:00:00


In [84]:
users_who_need_recs.shape

(31781,)

In [86]:
users_who_need_recs

array([    1,     2,     5, ..., 53964, 53966, 53968])

In [123]:
for user_id in users_who_need_recs:  
    print(user_id)

1
4
5
6
10
11
12
13
14
15
16
18
19
20
21
23
26
27
28
29
35
36
37
38
40
41
46
47
49
50
56
58
59
60
62
63
65
67
68
69
70
71
72
76
78
80
81
82
83
84
87
88
91
92
93
94
95
98
100
103
105
107
108
109
110
111
113
114
115
116
117
118
120
123
126
127
128
130
131
132
133
134
137
142
143
144
145
148
150
151
153
154
160
161
163
165
167
168
169
170
172
173
176
177
178
179
182
184
185
186
188
189
190
191
192
193
194
195
196
197
198
203
204
205
208
210
211
213
215
216
217
218
220
222
223
226
229
231
232
233
234
236
237
239
241
243
244
245
247
248
249
250
252
253
254
255
256
257
262
263
264
267
268
269
271
276
278
279
280
281
282
283
284
285
286
289
297
299
300
301
302
303
305
311
312
314
316
317
318
320
321
322
323
324
325
326
327
329
330
333
334
335
337
339
340
341
343
345
346
351
352
353
356
357
358
360
361
363
364
365
366
367
368
370
371
372
373
374
375
377
378
379
380
381
382
383
384
385
386
387
390
391
394
395
397
398
399
400
401
402
403
404
405
406
407
409
410
411
416
419
421
422
423
425
426
42

In [126]:
def find_similar_movies(movie_id):
    '''
    INPUT
    movie_id - a movie_id 
    OUTPUT
    similar_movies - an array of the most similar movies by title
    '''
    # find the row of each movie id
    
    # find the most similar movie indices - to start I said they need to be the same for all content
    similar = movie_matrix_df.loc[movie_id, movie_matrix_df.loc[movie_id] != 0] \
        .sort_values(ascending=False)
    
    # pull the movie titles based on the indices
    similar_movies = movies.loc[similar.index]['movie']
    
    return similar_movies
    
# You made this function in an earlier notebook - using again here    
def get_movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = list(movies[movies['movie_id'].isin(movie_ids)]['movie'])
   
    return movie_lst


def make_recs():
    '''
    INPUT
    None
    OUTPUT
    recs - a dictionary with keys of the user and values of the recommendations
    '''
    # Create dictionary to return with users and ratings
    recs = dict()
    
    # How many users for progress bar
    users_count = len(users_who_need_recs)
    
    # Create the progressbar
    widgets = ['Test: ', progressbar.Percentage(), ' ', 
               progressbar.Bar(marker='0',left='[',right=']'), ' ', ETA()] 

    pbar = ProgressBar(widgets=widgets, maxval=users_count)
    pbar.start()
    
    # For each user
    i = 0
    for user_id in users_who_need_recs:  
        
        # Update the progress bar
        i += 1
        pbar.update(i) #this adds a little symbol at each iteration

        # Pull only the reviews the user has seen
        
        
        # Look at each of the movies (highest ranked first), 
        # pull the movies the user hasn't seen that are most similar
        # These will be the recommendations - continue until 10 recs 
        # or you have depleted the movie list for the user



    # finish the progress bar
    pbar.finish() 
    
    return recs

In [128]:
recs = make_recs()

Test: 100% [0000000000000000000000000000000000000000000000000000] Time: 0:00:00


In [129]:
recs

{}

In [77]:
get_movie_names([8, 91])

['Edison Kinetoscopic Record of a Sneeze (1894)', 'Le manoir du diable (1896)']

In [ ]:
recs = make_recs()

### How Did We Do?

Now that you have made the recommendations, how did we do in providing everyone with a set of recommendations?

`4.` Use the cells below to see how many individuals you were able to make recommendations for, as well as explore characteristics about individuals for whom you were not able to make recommendations.  

In [ ]:
# Explore recommendations


In [ ]:
# Some characteristics of my content based recommendations
users_without_all_recs = #store user ids without recs
users_with_all_recs = # users with all recs here
no_recs = # users with no recommendations

print("There were {} users without all 10 recommendations we would have liked to have.".format(len(users_without_all_recs)))
print("There were {} users with all 10 recommendations we would like them to have.".format(len(users_with_all_recs)))
print("There were {} users with no recommendations at all!".format(len(no_recs)))

In [ ]:
# Closer look at individual user characteristics - this may help it was from an earlier notebook
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].isnull() == False].index.values

    return movies


movies_watched(189)

In [ ]:
# More exploring
print("Some of the movie lists for users without any recommendations include:")
# Just one of many questions you could continue exploring...

### Now What?  

Well, if you were really strict with your criteria for how similar two movies are (like I was initially), then you still have some users that don't have all 10 recommendations (and a small group of users who have no recommendations at all). 

As stated earlier, recommendation engines are a bit of an **art** and a **science**.  There are a number of things we still could look into - how do our collaborative filtering and content based recommendations compare to one another? How could we incorporate user input along with collaborative filtering and/or content based recommendations to improve any of our recommendations?  How can we truly gain recommendations for every user?

`5.` In this last step feel free to explore any last ideas you have with the recommendation techniques we have looked at so far.  You might choose to make the final needed recommendations using the first technique with just top ranked movies.  You might also loosen up the strictness in the similarity needed between movies.  Be creative and share your insights with your classmates!

In [ ]:
# Cells for exploring